In [1]:
%matplotlib notebook
import os
import h5py
import tensorflow as tf
from tensorflow import keras
from models_yolo import UNet, CallBacks, DataGenerator
import matplotlib.pyplot as plt
from csbdeep.utils import plot_history
import obspy
import shutil
import numpy as np
import numpy.ma as ma
""" Setting random seeds """
from models import seed

# TensorFlow
tf.random.set_seed(seed)

# Python
import random as python_random
python_random.seed(seed)

# NumPy (random number generator used for sampling operations)
rng = np.random.default_rng(seed)

In [5]:
with open('list_very_noisy_arrays.txt', 'r') as f:
    contents = f.read()

list_of_strings = contents.split('\n')
listing = [str(item) for item in list_of_strings if item]

In [6]:
# show the model trained on synthetic data
pretrainfile = os.path.join("save", "model_snr_1_big_model.h5")
model = keras.models.load_model(pretrainfile)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 11, 2048, 1  0           []                               
                                )]                                                                
                                                                                                  
 input_7 (InputLayer)           [(None, 11, 2048, 1  0           []                               
                                )]                                                                
                                                                                                  
 tf.math.multiply_5 (TFOpLambda  (None, 11, 2048, 1)  0          ['input_8[0][0]',                
 )                                                                'input_7[0][0]']          

                                                                                                  
 activation_63 (Activation)     (None, 11, 32, 32)   0           ['conv2d_65[0][0]']              
                                                                                                  
 gaussian_dropout_63 (GaussianD  (None, 11, 32, 32)  0           ['activation_63[0][0]']          
 ropout)                                                                                          
                                                                                                  
 max_pooling2d_19 (MaxPooling2D  (None, 11, 29, 32)  0           ['gaussian_dropout_63[0][0]']    
 )                                                                                                
                                                                                                  
 tf.compat.v1.nn.conv2d_18 (TFO  (None, 11, 8, 32)   0           ['max_pooling2d_19[0][0]']       
 pLambda) 

 activation_71 (Activation)     (None, 11, 32, 32)   0           ['conv2d_73[0][0]']              
                                                                                                  
 gaussian_dropout_71 (GaussianD  (None, 11, 32, 32)  0           ['activation_71[0][0]']          
 ropout)                                                                                          
                                                                                                  
 up_sampling2d_12 (UpSampling2D  (None, 11, 128, 32)  0          ['gaussian_dropout_71[0][0]']    
 )                                                                                                
                                                                                                  
 concatenate_12 (Concatenate)   (None, 11, 128, 48)  0           ['up_sampling2d_12[0][0]',       
                                                                  'gaussian_dropout_61[0][0]']    
          

In [7]:
# create an array of size (N_inputs,640,2048)
N_ch, N_t = (640,2048)
N_sub=11
data_reconstructions = np.zeros((N_ch,N_t))
cwd = os.getcwd()
data_dir = os.path.join(cwd, "reconstruction_snr_1_noisy_inputs_big_model")
# Loop for each input. eval sample each have a shape (640,2048)
for n in range(39):
    """ Prepare samples and masks """
    # shape (640,11,2048,1)
    eval_sample = np.load(listing[n])
    masks = np.ones((N_ch, N_sub, N_t, 1))
    eval_samples = np.zeros_like(masks)
    # gutter and mid are both 5
    gutter = N_sub // 2
    mid = N_sub // 2
    for i in range(gutter):
        # 3 masks are put to 0
        masks[i, i:i+3] = 0
        # one channel of eval_samples contains the 11 first channels of eval_sample. shape :(11,2048)
        eval_samples[i, :, :, 0] = eval_sample[:N_sub]
        # i goes from 5 to 634
    for i in range(gutter, N_ch - gutter):
        #goes from 0 to 629
        start = i - mid
        # goes from 11 to 640
        stop = i + mid + 1
        masks[i, mid-1:mid+2] = 0
        #start:stop has a shape of 11 => consistent with first for loop
        eval_samples[i, :, :, 0] = eval_sample[start:stop]

    for i in range(N_ch - gutter, N_ch):
        masks[i, N_sub-3:N_sub] = 0
        eval_samples[i, :, :, 0] = eval_sample[-N_sub:]
    """ Create J-invariant reconstructions """
    results = model.predict((eval_samples, masks))
    for i in range(gutter):
        data_reconstructions[i] = results[i,i,:,0]
    for i in range(gutter, N_ch - gutter):
        data_reconstructions[i] = results[i,mid,:,0]
    for i in range(N_ch - gutter, N_ch):
        data_reconstructions[i] = results[i,N_sub-2,:,0]     
    filename = f"reconstruction_{n}.h5"
    h5file = os.path.join(data_dir, filename)
    with h5py.File(h5file, "w") as h5f:
        h5f.create_dataset("strain_rate", data=data_reconstructions)

20/20 [==============================] - 4s 202ms/step
